# Actors relationship by realisator
The goal here is to visualize the relationship of actors by seeing if they worked with the same realisator

#### 1/Preparation des données

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations

nodes_df = pd.read_csv('data/nodes_1960_to_1980.csv')
edges_df = pd.read_csv('data/edges_1960_to_1980.csv')

print(nodes_df.head())
print(edges_df.head())

          Id  category           Label
0  nm0317813     actor      Manuel Gil
1  nm0243749     actor   Antonio Durán
2  nm0649892     actor      José Orjas
3  nm0649563  director  Juan de Orduña
4  nm0720019    writer   Enrique Reoyo
      Source     Target      Label  Weight
0  nm0243749  nm0317813  tt0031458       1
1  nm0317813  nm0649892  tt0031458       1
2  nm0317813  nm0649563  tt0031458       1
3  nm0317813  nm0720019  tt0031458       1
4  nm0317813  nm0524001  tt0031458       1


In [7]:
actors = nodes_df[nodes_df['category'] == 'actor']
directors = nodes_df[nodes_df['category'] == 'director']


# Filtrer les arêtes pour ne garder que celles reliant les acteurs aux réalisateurs
actor_director_edges = edges_df[
    edges_df['Source'].isin(actors['Id']) & edges_df['Target'].isin(directors['Id'])
]

In [8]:
# Construire une dictionnaire où chaque réalisateur est lié à une liste des acteurs ayant travaillé avec lui
director_to_actors = {}
for _, row in actor_director_edges.iterrows():
    director = row['Target']
    actor = row['Source']
    
    if director not in director_to_actors:
        director_to_actors[director] = set()
    director_to_actors[director].add(actor)

In [9]:
# Créer des arêtes entre acteurs ayant collaboré avec le même réalisateur
actor_pairs = {}
for actors_set in director_to_actors.values():
    # Créer toutes les paires possibles d'acteurs ayant travaillé avec le même réalisateur
    for actor1, actor2 in combinations(actors_set, 2):
        # Trier les paires pour éviter les doublons
        pair = tuple(sorted([actor1, actor2]))
        
        # Ajouter ou incrémenter le poids de la connexion
        if pair not in actor_pairs:
            actor_pairs[pair] = 0
        actor_pairs[pair] += 1

In [10]:
# Construire le graphe
G = nx.Graph()

# Ajouter les nœuds (uniquement les acteurs)
for actor_id in actors['Id']:
    actor_label = actors[actors['Id'] == actor_id]['Label'].values[0]
    G.add_node(actor_id, label=actor_label)

# Ajouter les arêtes avec le poids correspondant
for (actor1, actor2), weight in actor_pairs.items():
    G.add_edge(actor1, actor2, weight=weight)

In [ ]:
# Exporter les données pour Gephi
# Export des nœuds
nodes_export = pd.DataFrame([(n, G.nodes[n]['label']) for n in G.nodes], columns=['Id', 'Label'])
nodes_export.to_csv('gephi/gephi_actors_nodes.csv', index=False)

# Export des arêtes
edges_export = pd.DataFrame([(u, v, G[u][v]['weight']) for u, v in G.edges], columns=['Source', 'Target', 'Weight'])
edges_export.to_csv('gephi/gephi_actors_edges.csv', index=False)

print("Les fichiers CSV pour Gephi ont été exportés avec succès.")

Les fichiers CSV pour Gephi ont été exportés avec succès.
